### Inputs 

In [1]:
from io import StringIO
import sqlite3

import numpy as np
import pandas as pd

from workout_maker import WorkoutMaker

__sqlite__

In [2]:
# custom sqlite adapters because of datatype parameterization issues
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(pd.tslib.Timestamp, lambda val: str(val))
con = sqlite3.connect('workout.db', detect_types=sqlite3.PARSE_DECLTYPES)

#create database if needed
with con:
    cur = con.cursor() 
    cur.execute("CREATE TABLE IF NOT EXISTS dim_user(user_id INT, user_name STRING, email STRING)")
    cur.execute("CREATE TABLE IF NOT EXISTS dim_progression(user_id INT, prog_dict JSON)")
    cur.execute("CREATE TABLE IF NOT EXISTS one_rep_max(user_id INT, data_start_date TIMESTAMP, data_end_date TIMESTAMP, orm_dict JSON, publish_time TIMESTAMP)")
    cur.execute("CREATE TABLE IF NOT EXISTS accessory(user_id INT, me_name STRING, ae_name STRING, ae_weight FLOAT, sets INT, reps INT, publish_time TIMESTAMP)")
    cur.execute("CREATE TABLE IF NOT EXISTS pause_workout(user_id INT, pause_date TIMESTAMP DEFAULT '2999-12-31 23:59:59', pause_flag BOOLEAN DEFAULT False)")

__initialize module__

In [3]:
workout = WorkoutMaker(con, 'Matt Guan', 'mguan999@gmail.com')

Welcome back Matt Guan!


### __First Run Only__
Enter your workout inputs here..
* `accessory` : pandas df containing accessory workout weights
* `prog` : dict containing monthly progression weight for each main workout
* `orm` : dict containing one rep max weights for each main workout

<div class="alert alert-info">
  <strong>Note:</strong> You only need to run this part if it is your first run OR you want to update something.
</div>

In [4]:
s = '''
me_name	ae_name	ae_weight	sets	reps
deadlift	Rows	100	4	10
deadlift	Pullups	0	4	10
deadlift	Curls	30	4	10
deadlift	Back Extension	225	4	10
squat	Leg Press	290	4	10
squat	Lying Leg Curl	85	4	10
squat	Calf Raise	225	4	10
squat	Hip Abd/Add-uction	250	4	10
bench	Dips	0	4	10
bench	Incline D.Bell Press	45	4	10
bench	Cable Tricep Ext.	42.5	4	10
bench	Machine Pec Fly	100	4	10
ohp	Sitting D.Bell Press	50	5	5
ohp	Cable Rear Delt Fly	25	4	10
ohp	Machine Rear Delt Fly	70	4	10
ohp	Machine Lat Raise	60	4	10
'''
accessory = pd.read_csv(StringIO(s), sep='\t')

prog = {'squat':2.5 , 'deadlift':2.5, 'ohp':0, 'bench':2.5}
orm = {'squat':240 , 'deadlift':290, 'ohp':127.5, 'bench':182.5}

In [5]:
workout.set_accessory(accessory)

2018-06-16 22:07:23,850 | INFO | dataframe is valid - accessory populated


,me_name,ae_name,ae_weight,sets,reps,user_id,publish_time
0,deadlift,Rows,100.0,4,10,1,2018-06-16 22:07:23.840519
1,deadlift,Pullups,0.0,4,10,1,2018-06-16 22:07:23.840519
2,deadlift,Curls,30.0,4,10,1,2018-06-16 22:07:23.840519


In [6]:
workout.set_dim_prog(prog)

duplicate entry deleted
new entry loaded
2018-06-16 22:07:24,133 | INFO | dict is valid - dim_progression populated


,user_id,prog_dict
0,1,"{""squat"": 2.5, ""deadlift"": 2.5, ""ohp"": 0, ""ben..."


In [7]:
workout.set_one_rep_max(orm)

duplicate entry deleted
new entry loaded
2018-06-16 22:07:24,451 | INFO | dict is valid - one_rep_max overwitten


### Pull Workout

In [8]:
workout.main()

2018-06-16 22:07:25,728 | INFO | workout saved to ./lp-workout.html
